In [1]:
import requests
import json

# 1. Daum 증권 페이지에서 주가 크롤링

[daum 증권](http://finance.daum.net/)에서 보여주는 인기 검색 상위 10개 기업의 결과는, 이전처럼 HTML 문서를 분석하여 크롤링하는 방식을 사용할 수 없습니다.

웹 페이지가 API로부터 실시간으로 변하는 주식의 정보를 주기적으로 요청하여 표시하고 있기 때문입니다. 

웹에서 요청하는 서버에 직접 요청하여 json 데이터를 얻은 후 출력하겠습니다. 

In [2]:
# json 파일로부터 원하는 데이터를 추출한 리스트를 반환

custom_header = {
    'referer' : 'http://http://finance.daum.net/quotes/A048410#home',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }


def get_data():
    result = []
    url = "https://finance.daum.net/api/search/ranks?limit=10"
    req = requests.get(url, headers = custom_header)
    if req.status_code == requests.codes.ok:
        print("접속 성공")
        api_data = json.loads(req.text)
        stock_data = api_data['data']
        
        for stock in stock_data:
            result.append([stock['rank'], stock['name'], stock['tradePrice']])
        
    else:
        print('접속 실패')
        
    return result

def main():
    data = get_data()
    
    for d in data:
        print(d)

        
main()

접속 성공
[1, '셀트리온', 322000.0]
[2, '부광약품', 41550.0]
[3, '삼성전자', 54100.0]
[4, '신일제약', 58100]
[5, '한국전력', 19400.0]
[6, '현대차', 124500.0]
[7, '씨젠', 210900.0]
[8, '일양약품', 94000.0]
[9, '일신바이오', 3815.0]
[10, '두산중공업', 8610.0]


# 2. 네이버 실시간 검색어 크롤링

네이버 실시간 검색어와 연관 검색어를 크롤링하기
- `get_keyword_ranking` : 실시간 검색어 정보를 담고 있는 리스트

In [3]:
custom_header = {
    'referer' : 'https://www.naver.com/',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'  }


def get_keyword_ranking():
    
    result = []
    url = "https://apis.naver.com/mobile_main/srchrank/srchrank?frm=main&ag=30s&gr=2&ma=-2&si=-2&en=-2&sp=-2"
    req = requests.get(url, headers = custom_header)
    if req.status_code == requests.codes.ok:
        print("접속 성공")
        api_data = json.loads(req.text)
        ranking = api_data['data']
        
        for i in ranking:
            result.append([i['keyword'], i['keyword_synonyms']])
        
    else:
        print("접속 실패")
        
    return result
    
    
    
def main():
    result = get_keyword_ranking()
    i = 1
    for keyword, synonyms in result :
        if synonyms :
            print(f"{i}번째 검색어 : {keyword}, 연관검색어 : {synonyms}")
        else:
            print(f"{i}번째 검색어 : {keyword}")
        i += 1

main()

접속 성공
1번째 검색어 : 중국 샨샤댐, 연관검색어 : ['샨샤댐']
2번째 검색어 : 영진약품
3번째 검색어 : 우비
4번째 검색어 : 이스타항공
5번째 검색어 : 제일약품
6번째 검색어 : 장후 시간외 거래
7번째 검색어 : 메가공무원
8번째 검색어 : 신신제약, 연관검색어 : ['sk디스커버리', '일신바이오', '유바이오로직스']
9번째 검색어 : 랩지노믹스
10번째 검색어 : 경동제약
11번째 검색어 : 박수인
12번째 검색어 : 흥선대원군 화폐
13번째 검색어 : 토요일 날씨, 연관검색어 : ['강수량', '호우주의보', '장마']
14번째 검색어 : 기상청 레이더영상
15번째 검색어 : 허훈
16번째 검색어 : 송파 사랑교회, 연관검색어 : ['사랑교회', '송파구 사랑교회']
17번째 검색어 : 광원산업
18번째 검색어 : 골프장 여배우
19번째 검색어 : 아들의 여자
20번째 검색어 : 땡전


# 3. 음식점 리뷰 크롤링

음식점을 소개하고 추천하는 서비스를 제공하는 웹사이트인 "망고플레이트"의 데이터로 음식점 리뷰를 크롤링하기
<br>

1. 리뷰를 불러오는 API의 url에 접근하여 어떤 음식점의 url이 주어졌을 때 "해당 음식점의 리뷰"를 모두 불러오는 코드를 작성
2. 특정 키워드를 검색할 때 나타나는 음식점들의 href로 접근하여 여러 음식점들의 리뷰를 불러오는 코드를 작성

In [4]:
# 1. 리뷰를 불러오는 API의 url에 접근하여 어떤 음식점의 url이 주어졌을 때 "해당 음식점의 리뷰"를 모두 불러오는 코드를 작성

from bs4 import BeautifulSoup
import requests
import json

custom_header = {
    'referer' : 'https://www.mangoplate.com',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36' }

def get_reviews(code) : # code : 레스토랑 코드
    comments = []
    
    i = 0
    while True :
        url = f"https://stage.mangoplate.com/api/v5{code}/reviews.json?language=kor&device_uuid=\
        V3QHS15862342340433605ldDed&device_type=web&start_index={i}&request_count=50&sort_by=2"
        # 쿼리 부분
        # -- request_count = 50 : 50개씩 보겠다
        req = requests.get(url, headers = custom_header)



        if req.status_code == requests.codes.ok:    
            print("접속 성공")
            data = json.loads(req.text)
            
            if data:
                for d in data:
                    comments.append(d['comment']['comment'])
                i += 50
            else:
                break
        else:
            print("Error code")
            break
    
    return comments
    
    

def main() :
    href = "/restaurants/iMRRP69qtkeO"
    print(get_reviews(href))
    

if __name__ == "__main__" :
    main()


접속 성공
접속 성공
접속 성공
['맛, 분위기, 서비스 모두 너무 좋았어요 ☺️ ', '맛있다, 비릴법한 재료로 신선한 맛을 내는 게 신기하다.\n조용한 분위기도 좋고, 자주 올 듯 하다.', '“고등어샌드위치만 있는게 아니라구요”\n\n#츠지 조리전문학교\n프랑스 르꼬르동블루, 미국 CIA와 함께 세계 3대 조리학교로 불리는 츠지 조리전문학교. 국내 일식 씬 역시 츠지 출신 셰프들이 한 축을 담당하고 있는데, 내가 아는 곳만 해도 ‘카덴’, ‘하레’, ‘미나미’, ‘분노지’, ‘재패니즈다이닝안심’ 등이 있고 과연 음식 수준이 하나같이 훌륭하다.\n\n츠지 출신 셰프가 운영하는 곳 중 ‘미라이’를 빼놓을 수 없다. 츠지를 졸업한 이후 이치에와 아키에서 경험을 쌓은 두 셰프님들이 의기투합하여 만든 곳으로, 오픈 초기부터 사바산도로 선풍적인 인기를 끌고 지금까지도 그 인기가 식지않고 있다.\n\n#이자카야의 조건\n2000년대부터 국내에 개념이 잡히기 시작한 이자카야는 이제 갓포, 일식 다이닝 바, 가이세키로 층이 나뉘는 동시에 로바다야키, 야키토리, 쿠시아게, 텐푸라 등 영역이 세분화, 전문화되고 있다. 미각을 일깨우는 새로운 음식에 대한 요구는 날로 높아지고 있으며, 가게 입장에서는 손님들이 이 곳에 와야만 하는 이유를 만들어내는 것이 생존을 위한 필수 조건이 되었다.\n\n마포와 더불어 이자카야들의 대표적인 격전지인 압구정. 이치에, 갓포아키, 로만테이, 쿠이신보 같은 강자들이 득실대는 이 곳에서 미라이는 생존을 넘어 점차 강남을 대표하는 이자카야 중 하나로 자리잡고 있다. 젊고 유능한 두 셰프가 머리를 맛대고 만들어내는 참신한 요리들이 손님들로 하여금 이 곳을 방문할 수 밖에 없도록 만들고 있기 때문이다.\n\n#사바산도가 있다\n이치에에 고등어볶음밥과 멘치카츠가 있고, 로만테이에 마라 금태와 꼼장어 짚불구이가 있고, 쿠이신보에 볏짚 타다끼가 있다면 미라이에는 사바산도가 있다.\n\n‘밀도’에서 공수한 바삭하면서도 촉촉한 식빵, 은근하게 절여진 감칠맛나는

In [5]:
# 함수 실행하기 
# 당도[아이스크림 가게]
print(get_reviews("/restaurants/0my2hMzHlOYD"))

접속 성공
접속 성공
접속 성공
접속 성공
접속 성공
['사진으로 볼때는 아이스크림에 무슨 떡이 올라간건가 했었는데 맛보기 맛이었네용\n\n맛보기까지해서 소금, 자두, 구운 피스타치오, 복숭아, 바질우유, 참외, 쌀, 백향과 이렇게 8가지 맛을 다 맛봤습니다! 일단 자두가 너모 맛있었어오ㅠㅠ 참외도 생각보다 맛있었구요 지금 여름이라 그런지 상큼한 과일 맛이 너무 잘어울리고 시원했어요! \n\n자두도 진짜 과일이 갈아서 들어간건지 자두 껍질 과육이 보이더라구요ㅎㅎ 맛있었어요! 소금맛은 가로수길 우유니에서 너무 맛있게 먹어서 거기 보다는 좀더 달달한 맛이었어요 \n\n다시 간다면 저의 픽은 참외랑 자두! 매일 매일 맛이 달라지는 것도 좋고 회전이 빨라서 오래 안기다린 것도 좋았네요. 근데 다른 아이스크림이 훨씬 빨리 녹는 것 같기는 합니다ㅠ 빨리 녹아내려서 길가면서 옷에 다흘렸어요 휴지는 필수!! 로 챙겨서 나오세요', '우유니, 젤라띠젤라띠를 너무 좋아하는 1인으로서 설레이는 마음으로 들린 당도! 메인은 복숭아와 소금, 맛보기는 바질토바토와 백향과였어요. 바질토마토 한입먹고 와아아 감동했지만 백향과와 복숭아는 밍밍밍밍... 소금도 맛있었지만 기대가 너무 컸던 탓인지 그냥 그저 그랬...ㅠㅠ 특히 복숭아는 쿨피스에 물을 탄 맛처럼 느껴져서 좀 아쉬웠어요. 직원분들은 굉장히 친절하세요.', '-망원동 당도-\n\n당도를 왜 이제서야 가봤나 후회한 하루였어요ㅋㅋ\n망원동을 자주 갔어도 카페만 가봤지 사실 아이스크림 맛있어봤자라는 생각을 많이 했거든요.\n\n특이하게 맛 두개랑 샘플 두개해서 샘플은 작은 사이즈로 올려주시더라구요. 저는 흑임자, 천도복숭아, 백향과, 소금을 기본으로 하구 바질토마토는 샘플로 맛 봤어요!\n\n진짜 맛없는 맛이 없었고 제가 과일을 좋아해서 그런지 과일맛이 진짜 맛있었어요ㅠㅠ 조만간 또 가고 싶은 맛!', '한5번 정도 방문한 것 같다. 특별한 맛은 아니지만 정직한 맛 그리고 사장님 내외분이 친절하셔서 망원동에 들르면 꼭 가게되는곳이다. 소

In [12]:
# 2. 특정 키워드를 검색할 때 나타나는 음식점들의 href로 접근하여 여러 음식점들의 리뷰를 불러오는 코드를 작성

# 2-1) 특정 키워드를 검색했을 때 나타나는 음식점들의 hredf 추출
from bs4 import BeautifulSoup
import requests
import json            

custom_header = {
    'referer' : 'https://www.mangoplate.com/',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36' }

def get_restaurants(name):
    # 검색어가 들어왔을 때 결과로 나타나는 식당 리스트
    url = "https://www.mangoplate.com/search/" + name
    req = requests.get(url, headers = custom_header)
    soup = BeautifulSoup(req.text, "html.parser")
    divs = soup.find_all('div', class_="list-restaurant-item")
    
    result = []
    for div in divs:

        href = div.find('a').attrs['href']
        title = div.find('h2', class_='title').get_text().replace('\n','').replace(' ','')
        result.append((title, href))

    return result 


def main():
    name = input()
    rest_list = get_restaurants(name)
    
    for i in rest_list:
        print(i)

main()

초밥
('스시카나에', '/restaurants/UjiK7BUxljkP')
('미라이', '/restaurants/iMRRP69qtkeO')
('스시코에', '/restaurants/6deWyIABIlnt')
('스시아라타', '/restaurants/vZ74ooBDahfg')
('스시키', '/restaurants/ugq5zpw24LjX')
('타쿠미곤', '/restaurants/syba5M_Z_uYd')
('스시소라(광화문점)', '/restaurants/dlG6_fId6D2Y')
('스시이마', '/restaurants/B64oLGvWbo5d')
('스시려프리미엄', '/restaurants/iMwOS45hgNv0')
('고료리켄', '/restaurants/TznMMYOSqMQK')
('이요이요스시(파크자이3호점)', '/restaurants/QJH59JGDLYXj')
('우제홍참치(논현본점)', '/restaurants/GsHC7WCEjX')
('이요이요스시(공덕1호점)', '/restaurants/Thi7FYrxzA')
('스시나미키', '/restaurants/9TgE3uc7XPc6')
('스시쿤', '/restaurants/piPw4gb_M8')
('스시조(서울웨스틴조선호텔)', '/restaurants/tBo4Hr5boy')
('아리아께(서울신라호텔)', '/restaurants/WOnlBsE3wq')
('형제상회(노량진본점)', '/restaurants/drXX2BDsGZ')
('와슈다이닝슈토', '/restaurants/PsgSgblG4XdD')
('오가와', '/restaurants/ba-druB2Lm')


In [17]:
# 2-2) 추출한 음식점 href들의 상위 리뷰 5개씩 출력
custom_header = {
    'referer' : 'https://www.mangoplate.com/',
    'user-agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36' }

def get_restaurants(name):
    # 검색어가 들어왔을 때 결과로 나타나는 식당 리스트
    url = "https://www.mangoplate.com/search/" + name
    req = requests.get(url, headers = custom_header)
    soup = BeautifulSoup(req.text, "html.parser")
    divs = soup.find_all('div', class_="list-restaurant-item")
    
    result = []
    for div in divs:

        href = div.find('a').attrs['href']
        title = div.find('h2', class_='title').get_text().replace('\n','').replace(' ','')
        result.append((title, href))

    return result 

def get_reviews(code):
    comments = []
    
    url = f"https://stage.mangoplate.com/api/v5{code}/reviews.json?language=kor&device_uuid=vcOh715954905121191516cUAj1&device_type=web&start_index=0&request_count=5&sort_by=2"
    req = requests.get(url, headers = custom_header)
    if req.status_code == requests.codes.ok:
        print("접속 성공")
        
        data_api = json.loads(req.text)
        
        for data in data_api:
            comments.append(data['comment']['comment'].replace('\n', ''))
            
    return comments
    

def main():
    name = input()
    rest_list = get_restaurants(name)
    
    for title, href in rest_list:
        reviews = get_reviews(href)
        print(reviews)
        print("\n"*2)
        print("="*30)
        
        
main()
        

짜장면
접속 성공
['쥬에 인테리어 정말 짱이고 친절하고 맛도 괜찮았습니다!!단품으로 먹었는데 양이 많아서 좋았어욬ㅋㅋ 배터짐누릉지랑 새우창펀 저거 두개가 제일 맛있었어요!두개는 꼭 드셔보세요! 차도 하나 시킬걸 했어요 단품으로 많이 먹다보니까 자꾸 물과 차가 먹고싶더라고요', '런치 자작코스에 새우창펀 추가해서 먹었었어용전체적인 음식의 간도 괜찮았고 음식 구성도 좋았지만 가성비 측면에선 잘 모르겠는 .. 코스 치고 양도 좀 적은 편이고 .. 창펀 추가해서 그나마 적당히 먹었어요코스 중에는 기억에 남는 메뉴는 없고 그냥 새우창펀이 제일 맛있었네유 ', '조용하고 고급스러운 분위기라 특별한 날이나 손님 접대할 때 가기 좋은 곳. 코스보다는 먹고싶은 거 골라서 먹고싶어서 단품으로 주문했다.새우창펀, 흑초 탕수육, 양주 볶음밥, 광동식 가지 두부 이렇게 먹었는데 정말 다 맛있었다. 특히 가지요리 최고. 역시 가지는 중식 스타일로 먹어야 맛있다. 양이 적은 편이고 가격이 좀 있지만 그만한 가치가 있는 곳이다. ', '#한남동 #쥬예"현존 서울 최고의 딤섬"1. 한국에서 맛있는 딤섬을 찾기가 꽤 힘들다. 맛으로는 중국 본토와 다르고 풍성함에는 미국식 딤섬에도 밀린다. 게다가 가격은 왜이리 비싼지...조금 인기 있는 집은 가격 부담에 배터지게 딤섬을 먹기가 힘들다. 2. 그래서 그런지 가격 좋고 맛이 조금 있느면 금새 인기있는 딤섬집이 된다. 이대앞의 <미스터 서왕만두>, 왕십리의 <빠오즈푸>가 좋은 예일 것이다.3. 그런데 실제 계급장 다 떼고 맛으로만 비교를 하면 어디가 최고일까? 난 지금까지 먹어본 딤섬 중에 이곳 <쥬에>를 꼽고싶다.4. 4인 식사에 12만원이라는 예약금을 내고 예약을 할 때부터 조금 의심스럽긴 했지만 직접 방문해보니 분위기 빵빵의 고급 중식당이다. (이태원 스타일의 만두집인 줄 알았기에 ㅋ)여행 출발하는 길이라 츄리닝 바람에 운동화 신고간 내 모습이 조금 어색해 보일 정도 ㅎㅎ5. 넓직한 테이블과 쾌적하게 밝고 경치 좋은 2층 자리는 오붓하고 격

접속 성공
['군만두, 새우만두, 우롱차새우만두 호평이 많아 기대했는데 그닥..군만두가 빠삭하고 쫄깃하니 맛있음우롱차가 느끼함을 잡아주니 추천추천', '그리: 군만두 맛집. 다른건 그냥 그럼. 떠우장(콩국)은 나름 별미이니 드셔보세여. (4/5)쭈기: 기본도 못하는 집들 천지 (4.5/5)', '부산에서 만난 인생만두집! 앞으로 부산역 근처를 지나치게 되면 꼭 들를 것 같다.', '2박3일부산여행의 첫끼입니다망플후기보니 맛은 있는데 웨이팅하면서까지 먹을 맛은아니다가 지배적이네용??저는 웨이팅까지 안해서그런지 매우 만족^~^주문은 새우만두 군만두 고기만두 했고만족순서도 동일합니당고기만두는 사진없네요!코로나 시기에도 여러식당들을 가봤지만 거리두기 차원에서 테이블 띄어서 앉는데는 첨이네요 약간 감동.. 좌석도 몇개 없으면서..!ㅠ아 군만두 육즙이 겁나 튀어서 거리두기 안햇으면옆테이블 손님한테 튀엇을거같아요 유명한곳 치고 직원분들도 엄청 친절하시네요서울 노포갬성아시죠 다들? 욕쟁이할머니 직전급의 주인분들 더러 잇져ㅎ+아쉬운점첫끼로 군만두에 병맥 먹을 생각에 신났는데 주류 판매X제가 갔을땐 바로 앞에 도로가 공사중이여서 출입문도돌아가야하고 공사소음있었습니다', '평점 : 3.5 / 5맛있긴 한데 망플 평점 만큼은 아니다?물론 망플 평점이 만능은 아니겠지만 생각했던 것만큼 맛있지는 않았다. 웨이팅이 좀 길기도 했고, 서비스 또한 정신이 없으셔서 그랬는지 좋은 편은 아니었다. 물론 중국에서 이런저런 만두를 많이 먹어본 탓인지도 모르겠다.새우만두 하나만큼은 인정할 만하다. 딱 배어무는 순간부터 그 육즙이 장난아니게 감칠맛이 돈다. 하지만 비비고 교자 새우만두를 먹어봤는가? 그 맛과 비슷하여 굳이 기다리면서 먹어야 할까 하는 생각이 드는 맛이기도 했다.  튀김만두는 중국집에서 주는 양산형 튀김만두 그 이상도 이하도 아니였고, 고기만두는 그냥저냥 평범했다.부산역 근처라 그냥 호기심 삼아 가보기는 나쁘지 않다. 그러나 너무 큰 기대는 하지 않았으면 좋겠다.']



접속 성공
[

접속 성공
['세 명이서 꿔바로우, 양꼬치 3인분, 옥수수온면, 하얼빈맥주 시켰고 배불리 먹었습니당양꼬치 하나도 안 비리고꿔바로우도 쫄깃쫄깃했어요옥수수온면은 처음 먹어봤는데 약간 잔치국수느낌..?다른 데서 어떻게 나오는지는 모르겠지만만족스럽게 먹었어요! 재방문의사 200퍼센트', '8/10', '⭐⭐⭐⭐⭐어향육사, 마파두부, 볶음밥, 물만두는 서비스.양꼬치집 가서 양꼬치 안시키는 사람이 있다고요? 네 저요... 하지만 양꼬치가 아니더라도 진짜 실력있는 중국집은 요리가 더 맛있다구요... 그리고 여기가 그렇습니다.마파두부는 보통 싼맛에 시켜먹는데 여기는 정말 찐이어유. 마라의 냄새가 확 느껴지는 - 화자오 팍팍 - 고춧기름 가득 소스와 고기는 고기대로 두부는 두부대로 절묘하게 볶아진 진짜 본토 마파두부. 보통은 두부가 이리저리 흐물흐물해져있는데 과장 조금 보태 탱탱하다 싶을 정도로 조직감이 잘 살아있어요. 공기밥을 외치는 맛.어향육사는 약간 굴소스 맛 고기볶음? 단짠의 매력, 굴소스의 감칠맛, 그러면서 재료들의 풍미가 어우러진 볶음요리. 여기 볶음류를 참 잘해요. 맥주 홀짝이다보니 벌써 그릇이 바닥를 비우더라고요...탄수화물이 아쉬워 메뉴에는 없는데 볶음밥을 부탁드려봤습니다. 흔쾌이 ㅇㅋ하신 뒤 나온 볶음밥은 오아.. 계란볶음밥인데, 밥알이 계란에 코팅되서 은은한 노란빛, 너무 마르지도 질지도 않은 밥에 신기하게도 오이가 들어가 향긋함을 더해요. (호불호 포인트) 배불렀는데 역시 다 먹어버렸습니다.신기하게도 모든 테이블에 물만두를 서비스로 주셔요. 양고기향이 물씬 나는 육즙 가득한 물만두. 이쯤되면 감사함이 물씬...가게 벽면에는 안중근 의사의 명언이 붙어있습니다. 큰일을 이루려면 멀리 내다봐야 한다. 앞으로는 가까운 동네맛집만 생각하는게 아니라 먼 곳도 내다보고 찾아가봐야겠습니다,,,,', '잡내 진 짜 안남.고기 정말 맛있음양꼬치 2인분, 어향가지 시켰고 계속들어감 너모 마싯슴 흑흑서비스로 주는 만두 사진은 없지만 무시하면 안됨ㅋㅋㅋㅋ 속이 진짜 풍미 쩝니다

접속 성공
['사진만 봐도 눈물이 나네요... 하염없이 맛있습니다... 직원분의 유려한 중국식 한국말은 현지의 맛을 더욱 극대화해줍니다. 애매한 시간에 가서 웨이팅을 피해 보아요...:,)...', '마라샹궈(맛 : 웨이러 / 약간 매운맛)에 재료 넣어서 먹었는데요. 생각보다는 매워서 당황했어요. 맛은 있어요. 튀김 꽃빵은 꼭 드세요. 후회안하실 거에요', '몇 번을 갔는지 몰라요.인생 마라샹궈 + 튀김 꽃빵 조합이 미쳤습니다. 튀김꽃빵이 이 곳의 히든카드.근데 안 맵게 먹으면 별로예요. 맵게 먹어야 맛있음. 어떤 친구는 카레 맛이 난다고 하기도 .. 무튼 짱 맛있어요.재방문의사 200퍼 ', '마라샹궈 초보자들에게 추천 !!다른곳에 비해서 향신료맛이 약한편이고알맞은 간이 좋았어요 ! 너무 짠곳들에 호되게 당해서...알맞은 간에 감동하고 왔습니다 ..저는 옥수수면이 젤 마싯써서 옥수수면 왕창 넣어요!❤️고기도 신선하구요 서비스도 친절 ! 가격도 괜차나요그리고 꿔바로우 양도 많고 쫀득쫀득하고 맛있어요정말 많이갔는데 갈때마다 배터질때까지 먹고나와요..만두는 비추천해요 (한개먹고 손이안감)', '워낙 유명해서..ㅎㅎ애정마라샹궈는 실패 없이 갈때마다 맛있어요. 근데 가격이 막 저렴한 느낌은 전혀 아닙니당..이대 근처에도 맛있는 곳이 있는데, 거기서 2만원 안나왔던 양인데 이번에 애정에서 28000원인가 나왔어요!가격이 오른건가? 무튼 싼 느낌은 전혀 아니에욥그리고 기본맛이 있고, 땅콩소스 들어간 맛이 있는데땅콩소스 들어간게 훨 맛있으니 그걸로 시키세영이번에 까먹고 기본으로 시켰다가 약간 실망했어요..']



접속 성공
['전체적으로 다 맛있으나 누룽지 탕은 평범했다', '유린기랑 짬뽕이 존맛이었어요가지튀김은 기대에 못미쳤네요 연남동 하하가 훨 나아요', '맛있어요 오랜만에 너무 느끼하지않고 달짝새콤한만족스러운 크림새우였어요소고기가지덮밥도 깐풍기도 다 메뉴 성공이었어요다만 서비스가 불쾌할 정도로 꽝이었습니다 ..', '맛있는데 날을 약간 타는거 같은..? 짬뽕맛이 일